# DES 加密算法

In [25]:
# 定义明文与密钥
plain_text = '10503141'  # 64位明文
secret_key = '12345678'  # 64位密钥

if plain_text is None or len(plain_text) != 8:
    raise ValueError('明文必须为8字节字符串')
if secret_key is None or len(secret_key) != 8:
    raise ValueError('密钥必须为8字节字符串')
print(f'明文：{plain_text}')
print(f'密钥：{secret_key}')

明文：10503141
密钥：12345678


## 工具方法

In [26]:
def char2bin(c: str) -> list[int]:
    """字符转换为二进制数组"""
    return list(map(int, list(bin(ord(c))[2:].zfill(8))))

## 子密钥生成

In [27]:
# 密钥转换为二进制数组
key_bin = []
for char in secret_key:
    key_bin.extend(char2bin(char))
print(f'密钥原始二进制数组: {key_bin}')

密钥原始二进制数组: [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0]


In [28]:
# 置换选择1，抛弃了 8、16、24、32、40、48、56、64
PC1_MAP = [
    57, 49, 41, 33, 25, 17, 9,
    1, 58, 50, 42, 34, 26, 18,
    10, 2, 59, 51, 43, 35, 27,
    19, 11, 3, 60, 52, 44, 36,
    63, 55, 47, 39, 31, 23, 15,
    7, 62, 54, 46, 38, 30, 22,
    14, 6, 61, 53, 45, 37, 29,
    21, 13, 5, 28, 20, 12, 4
]
pc1 = [key_bin[i - 1] for i in PC1_MAP]
# print(f'置换选择1 (56位): {pc1}')
key_half_len = len(pc1) // 2
key_left = pc1[:key_half_len]
key_right = pc1[key_half_len:]
print(f'密钥左半部分: {key_left}')
print(f'密钥右半部分: {key_right}')

密钥左半部分: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
密钥右半部分: [0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]


In [29]:
# 循环左移函数
H_MAP = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]
h = lambda i: H_MAP[i]

# 置换选择2
# 抛弃了 9、18、22、25、35、38、43、54
PC2_MAP = [
    14, 17, 11, 24, 1, 5, 3, 28,
    15, 6, 21, 10, 23, 19, 12, 4,
    26, 8, 16, 7, 27, 20, 13, 2,
    41, 52, 31, 37, 47, 55, 30, 40,
    51, 45, 33, 48, 44, 49, 39, 56,
    34, 53, 46, 42, 50, 36, 29, 32
]

In [30]:
# 生成16个48位子密钥
sub_keys = []
for t in range(16):
    # 循环左移
    for _ in range(h(t)):
        key_left.append(key_left.pop(0))
        key_right.append(key_right.pop(0))
    # 置换选择2
    merge_key = key_left + key_right
    sub_key = [merge_key[i - 1] for i in PC2_MAP]
    sub_keys.append(sub_key)

for i in range(len(sub_keys)):
    print(f'子密钥{i + 1: 2}: {sub_keys[i]}')


子密钥 1: [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0]
子密钥 2: [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1]
子密钥 3: [1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0]
子密钥 4: [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1]
子密钥 5: [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1]
子密钥 6: [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0]
子密钥 7: [1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,

## 明文加密


In [31]:
# 明文转换为二进制数组
plain_bin = []
for char in plain_text:
    plain_bin.extend(char2bin(char))
print(f'明文原始二进制数组: {plain_bin}')


明文原始二进制数组: [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]


In [32]:
# 初始置换
IP_MAP = [
    58, 50, 42, 34, 26, 18, 10, 2,
    60, 52, 44, 36, 28, 20, 12, 4,
    62, 54, 46, 38, 30, 22, 14, 6,
    64, 56, 48, 40, 32, 24, 16, 8,
    57, 49, 41, 33, 25, 17, 9, 1,
    59, 51, 43, 35, 27, 19, 11, 3,
    61, 53, 45, 37, 29, 21, 13, 5,
    63, 55, 47, 39, 31, 23, 15, 7
]
ip = [plain_bin[i - 1] for i in IP_MAP]
# print(f'初始置换 (64位): {ip}')
plain_half_len = len(ip) // 2
plain_left = ip[:plain_half_len]
plain_right = ip[plain_half_len:]
print(f'初始置换左半部分: {plain_left}')
print(f'初始置换右半部分: {plain_right}')


初始置换左半部分: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1]
初始置换右半部分: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


In [33]:
# 扩展置换
E_MAP = [
    32, 1, 2, 3, 4, 5,
    4, 5, 6, 7, 8, 9,
    8, 9, 10, 11, 12, 13,
    12, 13, 14, 15, 16, 17,
    16, 17, 18, 19, 20, 21,
    20, 21, 22, 23, 24, 25,
    24, 25, 26, 27, 28, 29,
    28, 29, 30, 31, 32, 1
]
P_MAP = [
    16, 7, 20, 21, 29, 12, 28, 17,
    1, 15, 23, 26, 5, 18, 31, 10,
    2, 8, 24, 14, 32, 27, 3, 9,
    19, 13, 30, 6, 22, 11, 4, 25
]

# S盒
S_BOXS = [
    [
        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
    ],
    [
        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9],
    ],
    [
        [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
        [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
        [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
        [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12],
    ],
    [
        [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
        [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
        [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
        [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14],
    ],
    [
        [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
        [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
        [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
        [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3],
    ],
    [
        [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
        [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
        [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
        [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13],
    ],
    [
        [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
        [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
        [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
        [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12],
    ],
    [
        [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
        [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
        [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
        [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11],
    ]
]


def xor(a: list[int], b: list[int]) -> list[int]:
    """异或函数"""
    return [a[i] ^ b[i] for i in range(len(a))]


def f(text: list[int], key: list[int]) -> list[int]:
    """加密函数F"""
    # 扩展置换
    text_extended = [text[i - 1] for i in E_MAP]
    # 与子密钥异或
    text_extended = xor(text_extended, key)
    # S盒置换
    result = []
    for i in range(8):
        sub_text = text_extended[i * 6:(i + 1) * 6]
        y_bin = [sub_text[0], sub_text[-1]]  # 取出行号
        x_bin = sub_text[1:-1]  # 取出列号
        y_dec = int(''.join(map(str, y_bin)), 2)  # 转换为十进制
        x_dec = int(''.join(map(str, x_bin)), 2)  # 转换为十进制
        output_dec: int = S_BOXS[i][y_dec][x_dec]  # S盒置换
        output_bin: list[int] = list(map(int, bin(output_dec)[2:].zfill(4)))  # 转换为二进制
        result.extend(output_bin)
    # P盒置换
    return [result[i - 1] for i in P_MAP]

In [34]:
# 循环加密
old_left = plain_left
old_right = plain_right
for t in range(16):
    # 新的左半部分
    new_left = old_right
    # 新的右半部分
    new_right = xor(old_left, f(old_right, sub_keys[t]))

    old_left = new_left
    old_right = new_right

    print(f'第{t + 1}轮加密后的明文左半部分: {old_left}')
    print(f'第{t + 1}轮加密后的明文右半部分: {old_right}')
    print('-' * 50)


第1轮加密后的明文左半部分: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
第1轮加密后的明文右半部分: [0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1]
--------------------------------------------------
第2轮加密后的明文左半部分: [0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1]
第2轮加密后的明文右半部分: [1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1]
--------------------------------------------------
第3轮加密后的明文左半部分: [1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1]
第3轮加密后的明文右半部分: [0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0]
--------------------------------------------------
第4轮加密后的明文左半部分: [0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0]
第4轮加密后的明文右半部分: [0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1,

In [35]:
# 求ip_map的逆
IP_INVERSE_MAP = [0] * 64
for i in range(64):
    IP_INVERSE_MAP[IP_MAP[i] - 1] = i + 1

In [36]:
# 32位交换
plain = old_right + old_left
# 逆初始置换
plain = [plain[i - 1] for i in IP_INVERSE_MAP]
print('加密后的二进制密文: ', plain)
# 转换为字符串
secret = ''
for i in range(8):
    sub_plain = plain[i * 8:(i + 1) * 8]
    # 转换为十进制
    sub_plain_dec = int(''.join(map(str, sub_plain)), 2)
    # 转换为ASCII码
    secret += chr(sub_plain_dec)
print('加密后的字符串密文: ', secret)


加密后的二进制密文:  [1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1]
加密后的字符串密文:  Õ]bÌ÷¥»
